In [8]:
!unzip test_all.zip

Streaming output truncated to the last 5000 lines.
  inflating: test_all/48404765780136605.JPEG  
  inflating: test_all/5623874037695173.JPEG  
  inflating: test_all/8830923257791682.JPEG  
  inflating: test_all/5999784560329912.JPEG  
  inflating: test_all/641941689724576.JPEG  
  inflating: test_all/9636463870285614.JPEG  
  inflating: test_all/713318944677822.JPEG  
  inflating: test_all/4913615325728824.JPEG  
  inflating: test_all/008735498505776818.JPEG  
  inflating: test_all/3814762033873971.JPEG  
  inflating: test_all/23028839087812902.JPEG  
  inflating: test_all/8989128654123999.JPEG  
  inflating: test_all/6757518822057905.JPEG  
  inflating: test_all/725959409539307.JPEG  
  inflating: test_all/054779581564842195.JPEG  
  inflating: test_all/4833253932036542.JPEG  
  inflating: test_all/8826839449367463.JPEG  
  inflating: test_all/394024301579029.JPEG  
  inflating: test_all/6814583823502609.JPEG  
  inflating: test_all/3444018406073345.JPEG  
  inflating: test_all/02698

In [9]:
!unzip train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/monkey/n02488702_1549.JPEG  
  inflating: train/monkey/n02484473_925.JPEG  
  inflating: train/monkey/n02484473_1980.JPEG  
  inflating: train/monkey/n02488894_658.JPEG  
  inflating: train/monkey/n02484473_2389.JPEG  
  inflating: train/monkey/n02484473_1320.JPEG  
  inflating: train/monkey/n02484473_1446.JPEG  
  inflating: train/monkey/n02484473_1426.JPEG  
  inflating: train/monkey/n02488702_72.JPEG  
  inflating: train/monkey/n02488702_2650.JPEG  
  inflating: train/monkey/n02488702_4894.JPEG  
  inflating: train/monkey/n02488702_1992.JPEG  
  inflating: train/monkey/n02488702_4695.JPEG  
  inflating: train/monkey/n02484473_1574.JPEG  
  inflating: train/monkey/n02488702_3344.JPEG  
  inflating: train/monkey/n02488702_6259.JPEG  
  inflating: train/monkey/n02488702_5925.JPEG  
  inflating: train/monkey/n02484473_649.JPEG  
  inflating: train/monkey/n02488702_5615.JPEG  
  inflating: train/monkey/n02488702_6342.J

In [10]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.optim as optim

In [11]:
torch.cuda.set_device(0)
device = torch.device("cuda")

In [12]:
torch.manual_seed(42)
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

In [13]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(0.5),
     transforms.RandomVerticalFlip(0.5),
     transforms.RandomRotation(24),
     transforms.ToTensor(),
     transforms.RandomErasing(0.4),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

In [14]:
full_dataset = torchvision.datasets.ImageFolder("train", transform=transform)


In [15]:
classes = full_dataset.classes

In [16]:
train_size = int(0.8 * len(full_dataset))
validation_size = len(full_dataset) - train_size
trainset, validation_set = torch.utils.data.random_split(full_dataset, [train_size, validation_size])

In [17]:

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


In [18]:
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


In [19]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna = ekstrakcja cech
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=0)
        
        
        ## Warstwa max pooling = faktyczny klasyfikator w oparciu o cechy
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.pool2 = nn.MaxPool2d(2)  ### obrazki sie zmniejszaja, dopytac na kons
        self.fc1 = nn.Linear(32 * 13 * 13, 600) ### dopytac o to 16 x 5 x 5
        self.fc2 = nn.Linear(600, 150)
        self.fc3 = nn.Linear(150, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # mozna zawsze robic print(x.size()) i break, zeby wiedziec czemu 16x5x5
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [20]:
net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=5408, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=50, bias=True)
)

In [21]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         out_channels1 = 12
#         out_channels2 = 32
#         self.n_convs = 20
#         ## Warstwa konwolucyjna = ekstrakcja cech
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=out_channels1, kernel_size=5, stride=1, padding=0)
#         self.conv2 = nn.Conv2d(out_channels1, out_channels2, 5)

#         self.convs = [nn.Conv2d(in_channels=out_channels2, out_channels=out_channels2, kernel_size=5, stride=1, padding=2) for _ in range(self.n_convs)]
#         self.bns = [nn.BatchNorm2d(out_channels2) for _ in range(self.n_convs)]
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.pool2 = nn.MaxPool2d(2)
#         self.fc1 = nn.Linear(out_channels2 * 14 * 14, 2400)
#         self.fc2 = nn.Linear(2400, 600)
#         self.fc3 = nn.Linear(600, 50)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         for i in range(self.n_convs):
#             x = F.relu(self.convs[i](x))
#             x = self.bns[i](x)
#         x = self.pool1(x)
#         x = self.pool2(x)
#         x = torch.flatten(x, 1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [22]:
# net = Net().to(device)
# for conv in net.convs:
#     conv.to(device)
# for bn in net.bns:
#     bn.to(device)
# net

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [24]:
training_loss = 0
validation_loss = 0
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
  net.train()
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    training_loss += loss.item()
    
  net.eval()
  with torch.no_grad():
      for i, data in enumerate(validation_loader, 0):
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          loss = criterion(outputs, labels) 
          validation_loss += loss.item()

  if epoch and (epoch % 10 == 0 or epoch == num_epochs - 1):    
      print(f"Epoch: {epoch}, training loss: {training_loss/10:.3}, validation loss: {validation_loss/10:.3}")
      training_loss = 0
      validation_loss = 0

Epoch: 10, training loss: 1.54e+03, validation loss: 3.91e+02
Epoch: 20, training loss: 1.19e+03, validation loss: 3.23e+02
Epoch: 30, training loss: 1.11e+03, validation loss: 3.17e+02
Epoch: 40, training loss: 1.06e+03, validation loss: 3.18e+02
Epoch: 50, training loss: 1.02e+03, validation loss: 3.2e+02
Epoch: 60, training loss: 9.89e+02, validation loss: 3.2e+02
Epoch: 70, training loss: 9.65e+02, validation loss: 3.22e+02
Epoch: 80, training loss: 9.45e+02, validation loss: 3.24e+02
Epoch: 90, training loss: 9.26e+02, validation loss: 3.24e+02
Epoch: 99, training loss: 8.2e+02, validation loss: 2.96e+02


In [25]:
torch.save(net.state_dict(), 'model.tar.gz')

In [26]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
acc_sum = 0
with torch.no_grad():
    for i, data in enumerate(validation_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels) 
        validation_loss += loss.item()
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
          if label == prediction:
              correct_pred[classes[label]] += 1
          total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    acc_sum += accuracy

    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

Accuracy for class acoustic is: 23.7 %
Accuracy for class antenna is: 33.6 %
Accuracy for class bacteria is: 45.6 %
Accuracy for class battery is: 29.3 %
Accuracy for class bean  is: 32.5 %
Accuracy for class beetle is: 53.6 %
Accuracy for class bicycle is: 47.1 %
Accuracy for class birch is: 38.8 %
Accuracy for class bird  is: 24.2 %
Accuracy for class bomb  is: 39.6 %
Accuracy for class bread is: 33.9 %
Accuracy for class bridge is: 40.2 %
Accuracy for class camera is: 42.5 %
Accuracy for class carbon is: 24.5 %
Accuracy for class cat   is: 45.3 %
Accuracy for class corn  is: 34.2 %
Accuracy for class crab  is: 22.9 %
Accuracy for class crocodilian is: 25.7 %
Accuracy for class echinoderm is: 29.7 %
Accuracy for class egg   is: 24.4 %
Accuracy for class elephant is: 44.9 %
Accuracy for class fish  is: 43.2 %
Accuracy for class flower is: 80.2 %
Accuracy for class frog  is: 34.6 %
Accuracy for class fungus is: 47.4 %
Accuracy for class gauge is: 31.9 %
Accuracy for class hammer is: 62

In [27]:
acc_average_all_classes = acc_sum / len(classes)
acc_average_all_classes

39.068295725823255